# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [40]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [2]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")

# Display sample data
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Clouds,Wind,Weather Description
0,0,Barrow,US,71.2906,-156.7887,8.62,78,100,16.11,light snow
1,1,Ninghai,CN,29.2892,121.4247,50.65,65,16,5.66,few clouds
2,2,Atuona,PF,-9.8000,-139.0333,78.64,73,10,18.97,clear sky
3,3,Ushuaia,AR,-54.8000,-68.3000,38.86,87,75,3.44,light rain
4,4,Fortuna,US,40.5982,-124.1573,52.14,71,100,17.27,overcast clouds
5,5,Vaini,TO,-21.2000,-175.2000,77.16,94,100,3.44,moderate rain
6,6,Lagoa,PT,39.0500,-27.9833,59.61,78,10,20.65,clear sky
7,7,Ahipara,NZ,-35.1667,173.1667,68.20,97,100,13.69,overcast clouds
8,8,Punta Arenas,CL,-53.1500,-70.9167,50.11,93,40,39.12,light rain
9,9,Les Cayes,HT,18.2000,-73.7500,71.20,85,4,5.73,clear sky


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [3]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [6]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[
    (city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)
]

# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Clouds,Wind,Weather Description
2,2,Atuona,PF,-9.8000,-139.0333,78.64,73,10,18.97,clear sky
5,5,Vaini,TO,-21.2000,-175.2000,77.16,94,100,3.44,moderate rain
11,11,East London,ZA,-33.0153,27.9116,75.60,68,96,17.83,overcast clouds
22,22,Rikitea,PF,-23.1203,-134.9692,78.19,78,100,15.68,light rain
24,24,Busselton,AU,-33.6500,115.3333,87.71,32,44,3.98,scattered clouds
26,26,Fare,PF,-16.7000,-151.0167,81.59,76,100,16.67,light rain
27,27,Port Elizabeth,ZA,-33.9180,25.5701,75.09,88,75,19.57,broken clouds
34,34,Manokwari,ID,-0.8667,134.0833,83.03,76,91,7.81,overcast clouds
38,38,Lamu,KE,-2.2717,40.9020,88.02,73,100,15.57,light rain
41,41,Mahebourg,MU,-20.4081,57.7000,81.01,63,63,10.45,broken clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [7]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Clouds,Wind,Weather Description
2,2,Atuona,PF,-9.8000,-139.0333,78.64,73,10,18.97,clear sky
5,5,Vaini,TO,-21.2000,-175.2000,77.16,94,100,3.44,moderate rain
11,11,East London,ZA,-33.0153,27.9116,75.60,68,96,17.83,overcast clouds
22,22,Rikitea,PF,-23.1203,-134.9692,78.19,78,100,15.68,light rain
24,24,Busselton,AU,-33.6500,115.3333,87.71,32,44,3.98,scattered clouds
26,26,Fare,PF,-16.7000,-151.0167,81.59,76,100,16.67,light rain
27,27,Port Elizabeth,ZA,-33.9180,25.5701,75.09,88,75,19.57,broken clouds
34,34,Manokwari,ID,-0.8667,134.0833,83.03,76,91,7.81,overcast clouds
38,38,Lamu,KE,-2.2717,40.9020,88.02,73,100,15.57,light rain
41,41,Mahebourg,MU,-20.4081,57.7000,81.01,63,63,10.45,broken clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [11]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng
2,Atuona,PF,78.64,clear sky,-9.8000,-139.0333
5,Vaini,TO,77.16,moderate rain,-21.2000,-175.2000
11,East London,ZA,75.60,overcast clouds,-33.0153,27.9116
22,Rikitea,PF,78.19,light rain,-23.1203,-134.9692
24,Busselton,AU,87.71,scattered clouds,-33.6500,115.3333
26,Fare,PF,81.59,light rain,-16.7000,-151.0167
27,Port Elizabeth,ZA,75.09,broken clouds,-33.9180,25.5701
34,Manokwari,ID,83.03,overcast clouds,-0.8667,134.0833
38,Lamu,KE,88.02,light rain,-2.2717,40.9020
41,Mahebourg,MU,81.01,broken clouds,-20.4081,57.7000


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [12]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Atuona,PF,78.64,clear sky,-9.8000,-139.0333,
5,Vaini,TO,77.16,moderate rain,-21.2000,-175.2000,
11,East London,ZA,75.60,overcast clouds,-33.0153,27.9116,
22,Rikitea,PF,78.19,light rain,-23.1203,-134.9692,
24,Busselton,AU,87.71,scattered clouds,-33.6500,115.3333,
26,Fare,PF,81.59,light rain,-16.7000,-151.0167,
27,Port Elizabeth,ZA,75.09,broken clouds,-33.9180,25.5701,
34,Manokwari,ID,83.03,overcast clouds,-0.8667,134.0833,
38,Lamu,KE,88.02,light rain,-2.2717,40.9020,
41,Mahebourg,MU,81.01,broken clouds,-20.4081,57.7000,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [36]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [43]:
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = hotel_df["Lat"].loc[index]
    longitude = hotel_df["Lng"].loc[index]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    print(json.dumps(name_address, indent = 4, sort_keys = True))

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -139.0263047,
                    -9.804279899897706
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Pearl Resort",
                "address_line2": "Acc\u00e8s priv\u00e9, Hiva Oa, France",
                "categories": [
                    "accommodation",
                    "accommodation.hotel"
                ],
                "city": "Hiva Oa",
                "country": "France",
                "country_code": "fr",
                "county": "Marquesas Islands",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "name": "Pearl Resort",
                        "osm_id": 3206110908,
                        "osm_type": "n",
         

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -134.96598839892525,
                    -23.127535291946714
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Chez Bianca & Benoit",
                "address_line2": "Chemin des S\u0153urs, 98755 Gambier, France",
                "categories": [
                    "accommodation",
                    "accommodation.hotel"
                ],
                "city": "Gambier",
                "country": "France",
                "country_code": "fr",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "name": "Chez Bianca & Benoit",
                        "osm_id": 751195405,
                        "osm_type": "w",
                        "to

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    134.0774953932912,
                    -0.8648244162222687
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Hotel Arfak",
                "address_line2": "Jalan Brawijaya, Manokwari 98311, West Papua, Indonesia",
                "categories": [
                    "accommodation",
                    "accommodation.hotel",
                    "building",
                    "building.accommodation"
                ],
                "city": "Manokwari",
                "country": "Indonesia",
                "country_code": "id",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "building": "yes",
                        "internet_access": 

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    40.9033165,
                    -2.2711765997976454
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Hotel Archipelago villa bed and breakfast",
                "address_line2": "Corniche Path, Lamu, Kenya",
                "categories": [
                    "accommodation",
                    "accommodation.hotel"
                ],
                "country": "Kenya",
                "country_code": "ke",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "name": "Hotel Archipelago villa bed and breakfast",
                        "osm_id": 2618414049,
                        "osm_type": "n",
                        "tourism": "hotel"
    

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    57.69236426045249,
                    -20.408946222965504
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Grand Bel Air",
                "address_line2": "Deux Bras \u2013 Cent Gaulettes Road, Beau Vallon, Mauritius",
                "categories": [
                    "accommodation",
                    "accommodation.hotel",
                    "building",
                    "building.accommodation"
                ],
                "city": "Beau Vallon",
                "country": "Mauritius",
                "country_code": "mu",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "building": "yes",
                        "name": "G

...

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    98.7766691,
                    1.7409198998145303
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Hotel Prima Indah",
                "address_line2": "Jalan Brigjen Katamso n. 45, Pasar Baru, Sibolga, North Sumatra, Indonesia",
                "categories": [
                    "accommodation",
                    "accommodation.hotel"
                ],
                "city": "Sibolga",
                "country": "Indonesia",
                "country_code": "id",
                "county": "City of Sibolga",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "addr:housenumber": "n. 45",
                        "addr:street": "Jalan Brigj

{
    "features": [],
    "type": "FeatureCollection"
}
{
    "features": [],
    "type": "FeatureCollection"
}
{
    "features": [],
    "type": "FeatureCollection"
}
{
    "features": [],
    "type": "FeatureCollection"
}
{
    "features": [],
    "type": "FeatureCollection"
}
{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    30.479555299999994,
                    -30.71129979932944
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "The Spot Backpackers'",
                "address_line2": "Ambleside Road, Hibiscus Coast Ward 12, Hibiscus Coast Local Municipality, 4241, South Africa",
                "categories": [
                    "accommodation",
                    "accommodation.hotel"
                ],
                "city": "Hibiscus Coast Local Municipality",
                "country": "South Africa",
                "country_code": "za"

{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    27.61616575984849,
                    2.7750214708032734
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "H\u00f4tel Magbetu",
                "address_line2": "RN25, Isiro, Democratic Republic of the Congo",
                "categories": [
                    "accommodation",
                    "accommodation.hotel"
                ],
                "city": "Isiro",
                "country": "Democratic Republic of the Congo",
                "country_code": "cd",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "name": "H\u00f4tel Magbetu",
                        "osm_id": 226881789,
                        "osm_type": "w",
         

{
    "features": [],
    "type": "FeatureCollection"
}
{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    23.448324499999995,
                    -7.0093825997115315
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "H\u00f4tel KAMAS",
                "address_line2": "RN40, Mwene-Ditu, Democratic Republic of the Congo",
                "categories": [
                    "accommodation",
                    "accommodation.hotel"
                ],
                "city": "Mwene-Ditu",
                "country": "Democratic Republic of the Congo",
                "country_code": "cd",
                "county": "Kabinda",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "name": "H\u00f4tel KA

{
    "features": [],
    "type": "FeatureCollection"
}
{
    "features": [
        {
            "geometry": {
                "coordinates": [
                    -81.24808859999999,
                    19.279037800333068
                ],
                "type": "Point"
            },
            "properties": {
                "address_line1": "Turtle Nest Inn",
                "address_line2": "Bodden Town Road, Bodden Town KY1-1105, Cayman Islands",
                "categories": [
                    "accommodation",
                    "accommodation.hotel"
                ],
                "city": "Bodden Town",
                "country": "Cayman Islands",
                "country_code": "ky",
                "datasource": {
                    "attribution": "\u00a9 OpenStreetMap contributors",
                    "license": "Open Database Licence",
                    "raw": {
                        "addr:city": "South Cayman Palms",
                        "addr:country":

In [44]:
# Iterate through the hotel DataFrame

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = hotel_df["Lat"].loc[index]
    longitude = hotel_df["Lng"].loc[index]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["address_line1"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Atuona - nearest hotel: Pearl Resort
Vaini - nearest hotel: Keleti International Resort
East London - nearest hotel: Inverleith Terrace
Rikitea - nearest hotel: Chez Bianca & Benoit
Busselton - nearest hotel: No hotel found
Fare - nearest hotel: hôtel Maitai Lapita Village
Port Elizabeth - nearest hotel: Waterford Hotel
Manokwari - nearest hotel: Hotel Arfak
Lamu - nearest hotel: Hotel Archipelago villa bed and breakfast
Mahebourg - nearest hotel: Grand Bel Air
Luganville - nearest hotel: Beachfront Resort
Avarua - nearest hotel: Paradise Inn
Idenao - nearest hotel: No hotel found
Alofi - nearest hotel: No hotel found
Quatre Cocos - nearest hotel: Horisun
Carnarvon - nearest hotel: No hotel found
Vryburg - nearest hotel: International Hotel
Butaritari - nearest hotel: No hotel found
Sambava - nearest hotel: Royal Sava Hôtel
Cartagena - nearest hotel: Hotel Bellamar
Jejuri - nearest hotel: hotel durga lunch home
Karratha - nearest hotel: Karratha International Hote

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Atuona,PF,78.64,clear sky,-9.8000,-139.0333,Pearl Resort
5,Vaini,TO,77.16,moderate rain,-21.2000,-175.2000,Keleti International Resort
11,East London,ZA,75.60,overcast clouds,-33.0153,27.9116,Inverleith Terrace
22,Rikitea,PF,78.19,light rain,-23.1203,-134.9692,Chez Bianca & Benoit
24,Busselton,AU,87.71,scattered clouds,-33.6500,115.3333,No hotel found
...,...,...,...,...,...,...,...
702,Isiro,CD,85.03,scattered clouds,2.7739,27.6160,Hôtel Magbetu
705,Forbes,AU,85.91,clear sky,-33.3833,148.0167,No hotel found
707,Mwene-Ditu,CD,77.00,scattered clouds,-7.0000,23.4500,Hôtel KAMAS
710,Pandan,PH,78.21,moderate rain,14.0453,124.1694,No hotel found


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [65]:
# Drop the rows where there is no Hotel Name.
for index, row in hotel_df.iterrows():
    if hotel_df["Hotel Name"].loc[index] == "No hotel found":
        hotel_df = hotel_df.drop(index)
    

# Display sample data
hotel_df

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
2,Atuona,PF,78.64,clear sky,-9.8000,-139.0333,Pearl Resort
5,Vaini,TO,77.16,moderate rain,-21.2000,-175.2000,Keleti International Resort
11,East London,ZA,75.60,overcast clouds,-33.0153,27.9116,Inverleith Terrace
22,Rikitea,PF,78.19,light rain,-23.1203,-134.9692,Chez Bianca & Benoit
26,Fare,PF,81.59,light rain,-16.7000,-151.0167,hôtel Maitai Lapita Village
...,...,...,...,...,...,...,...
690,Port Shepstone,ZA,77.45,clear sky,-30.7414,30.4550,The Spot Backpackers'
700,Puerto Colombia,CO,77.09,clear sky,10.9878,-74.9547,Hotel Puerto Colombia
702,Isiro,CD,85.03,scattered clouds,2.7739,27.6160,Hôtel Magbetu
707,Mwene-Ditu,CD,77.00,scattered clouds,-7.0000,23.4500,Hôtel KAMAS


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [67]:
# Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label = "City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [77]:
# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 350,
    size = "Max Temp",
    scale = 0.5,
    color = "City",
    hover_cols = ["Country", "Weather Description", "Hotel Name"]
)
# Display the map
map_plot

######## Figure out how to add more things to each point (country code, weather description, etc.)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Max Temp,Country,Weather Description,Hotel Name)